# BjjHeroes

bjjheroes.com is a website with more up to date match data than flograppling, so it would be useful to scrape data from here.

In [256]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
import time
import sqlite3
# HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

## Get All athletes from list
link: https://www.bjjheroes.com/a-z-bjj-fighters-list

#soup.find_all('tbody')

#First Name
soup.find_all('td',class_='column-1')

#Last Name
soup.find_all('td',class_='column-2')

#Nickname
soup.find_all('td',class_='column-3')

#Team
soup.find_all('td',class_='column-4')


In [257]:
r = requests.get('https://www.bjjheroes.com/a-z-bjj-fighters-list')
soup = BeautifulSoup(r.text, 'html.parser')

new = soup.find_all('tbody')
new = new[0]
Athletes = pd.DataFrame(index=range(10000),columns=['First','Last','Nickname','Team','Link','ID'])
row = 0
for n in new.find_all('td', class_ = 'column-1'):
    #print(n('a'))
    href = n('a')[0].get('href')
    first = n('a')[0].get_text()
    Athletes.iloc[row,0] = first
    Athletes.iloc[row,4] = href
    Athletes.iloc[row,5] = href.split('=')[-1]
    row = row +1

row = 0
for n in new.find_all('td', class_ = 'column-2'):
    last = n('a')[0].get_text()
    Athletes.iloc[row,1] = last
    row = row +1
try:
    row = 0
    for n in new.find_all('td', class_ = 'column-3'):
        nickname = n('a')[0].get_text()
        Athletes.iloc[row,2] = nickname
        row = row +1
    
    row = 0
    for n in new.find_all('td', class_ = 'column-4'):
        team = n('a')[0].get_text()
        Athletes.iloc[row,3] = team
        row = row +1
except:
    pass

# Drop Missing Rows
Athletes = Athletes.dropna(axis=0,how='all')

# Make Link Searchable
Athletes['Link'] = 'https://www.bjjheroes.com' + Athletes['Link']
# Combine Name
Athletes['Name'] = Athletes['First'] + ' ' + Athletes['Last']

In [258]:
Athletes.tail()

,First,Last,Nickname,Team,Link,ID,Name
1331,Valdir,Canuto,NaN,NaN,https://www.bjjheroes.com/?p=7505,7505,Valdir Canuto
1332,Nakapan,Phungephorn,NaN,NaN,https://www.bjjheroes.com/?p=7512,7512,Nakapan Phungephorn
1333,Eliot,Kelly,NaN,NaN,https://www.bjjheroes.com/?p=7519,7519,Eliot Kelly
1334,Mauricio,Pereira,NaN,NaN,https://www.bjjheroes.com/?p=7556,7556,Mauricio Pereira
1335,Vinicius,Garcia,NaN,NaN,https://www.bjjheroes.com/?p=7636,7636,Vinicius Garcia


athlete info

r = requests.get('https://www.bjjheroes.com/bjj-fighters/andrew-wiltse')

soup = BeautifulSoup(r.text, 'html.parser')

print(soup.title.get_text().split('|')[0]) # Name

In [259]:
def get_matches(link):
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'html.parser')
        new = soup.find('tbody')
        
        Athlete_Name = soup.title.get_text().split('|')[0]
        
        Columns = ['Match_ID','Athlete_Name','Athlete_ID','Opponent_Name','Opponent_ID','W/L','Method','Event','Weight_Class','Year']
        Athlete_Matches = pd.DataFrame(columns=Columns,index=range(0,1000))

        row = 0
        try:

            for line in new.find_all('tr'):
                Athlete_Matches.iloc[row,0] = (line('td')[0].get_text()) # match ID
                Athlete_Matches.iloc[row,3] = (line('td')[1]('span')[0].get_text()) # Opponent Name
                try:
                    Opponent_ID_long = (line('td')[1]('a')[0].get('href'))
                    Opponent_ID_short = Opponent_ID_long.split('=')[-1]
                    Athlete_Matches.iloc[row,4] = Opponent_ID_short
                except:
                    pass
                Athlete_Matches.iloc[row,5] = (line('td')[2].get_text()) # W/L
                Athlete_Matches.iloc[row,6] = (line('td')[3].get_text()) # Method
                Athlete_Matches.iloc[row,7] = (line('td')[4].get_text()) # Event
                Athlete_Matches.iloc[row,8] = (line('td')[5].get_text()) # Weight Class
                Athlete_Matches.iloc[row,9] = (line('td')[7].get_text()) # Year
                row = row +1

            
            Athlete_Matches = Athlete_Matches.dropna(axis=0,how='all')
            Athlete_Matches['Athlete_Name'] = Athlete_Name
            Athlete_Matches['Athlete_ID'] = link.split('=')[-1]
            return(Athlete_Matches)
        except:
            pass
    except:
        pass
    

In [260]:
get_matches('https://www.bjjheroes.com/?p=12950')

,Match_ID,Athlete_Name,Athlete_ID,Opponent_Name,Opponent_ID,W/L,Method,Event,Weight_Class,Year
0,27910,Cole Abate,12950,Gabriel Sousa,9051,L,Referee Decision,WNO Champ.,70KG,2021
1,35004,Cole Abate,12950,Fabricio Andrey,11497,L,Referee Decision,ADCC,66KG,2022
2,27908,Cole Abate,12950,Geo Martinez,4996,W,Referee Decision,WNO Champ.,70KG,2021
3,28765,Cole Abate,12950,Miles Williams,NaN,W,Teepe choke,ADCC EC Trials,66KG,2021
4,28777,Cole Abate,12950,S. Weisenburg,NaN,W,Armbar,ADCC EC Trials,66KG,2021
5,28791,Cole Abate,12950,Kelly Quinn,NaN,W,Kimura,ADCC EC Trials,66KG,2021
6,28800,Cole Abate,12950,Jon Calestine,8767,W,Pts: 3x0,ADCC EC Trials,66KG,2021
7,28807,Cole Abate,12950,Estevan Martinez,12120,W,Pts: 3x0,ADCC EC Trials,66KG,2021
8,28810,Cole Abate,12950,Deandre Corbe,9934,W,Pts: 2x0,ADCC EC Trials,66KG,2021
9,28811,Cole Abate,12950,Keith Krikorian,12491,W,Pts: 10x0,ADCC EC Trials,66KG,2021


In [261]:
# Initialize an empty list to store DataFrames
all_dataframes = []

# Iterate through the list of athlete IDs
for link in Athletes['Link']:
    matches = get_matches(link)
    # time.sleep(1)
    #print('Working on: ' ,link)
    
    # Append the DataFrame to the list
    all_dataframes.append(matches)

master_df = pd.concat(all_dataframes, ignore_index=True)
master_df.tail()

,Match_ID,Athlete_Name,Athlete_ID,Opponent_Name,Opponent_ID,W/L,Method,Event,Weight_Class,Year
44692,21987,Vinicius Garcia,7636,Cody Heller,NaN,W,N/A,Atlanta SM Open,ABS,2019
44693,23307,Vinicius Garcia,7636,Daniel Olivier,NaN,W,Canto choke,New Orleans Open,88KG,2020
44694,23315,Vinicius Garcia,7636,Joshua Murdock,NaN,W,Points,New Orleans Open,ABS,2020
44695,23981,Vinicius Garcia,7636,Kyle Raemisch,NaN,W,Mounted X choke,F2W 153,85KG,2020
44696,24175,Vinicius Garcia,7636,Kevin Vieira,NaN,W,Hashimoto choke,Pan American,82KG,2020


In [262]:
# Connect to sql database & add to existing table
con = sqlite3.connect("bjj_network.db")
cur = con.cursor()
master_df.to_sql('Match_List_BjjHeroes', con, if_exists='replace', index=False)
con.commit()